# Lab Program 7 

Write a program to construct a Bayesian network considering medical data. Use this model to demonstrate the diagnosis of heart patients using standard Heart Disease Data Set. You can use Java/Python ML library classes/API 

In [1]:
#Diagnosis of heart patients using standard Heart Disease Data Set
import numpy as np
from urllib.request import urlopen
import pandas as pd

In [2]:
from google.colab import files 
uploaded = files.upload()

Saving 7.csv to 7.csv


In [3]:
names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'heartdisease']
heartDisease = pd.read_csv('7.csv', names = names)
heartDisease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heartdisease
0,28,1,2,130,132,0,2,185,0,0.0,?,?,?,0
1,29,1,2,120,243,0,0,160,0,0.0,?,?,?,0
2,29,1,2,140,?,0,0,170,0,0.0,?,?,?,0
3,30,0,1,170,237,0,1,170,0,0.0,?,?,6,0
4,31,0,2,100,219,0,1,150,0,0.0,?,?,?,0


In [4]:
pip install pgmpy

     |████████████████████████████████| 327kB 5.8MB/s 


In [5]:
#Modeling Heart Disease Data
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator

model = BayesianModel([('age', 'trestbps'), ('age', 'fbs'), ('sex', 'trestbps'), ('sex', 'trestbps'), 
                       ('exang', 'trestbps'),('trestbps','heartdisease'),('fbs','heartdisease'),
                      ('heartdisease','restecg'),('heartdisease','thalach'),('heartdisease','chol')])

# Learing CPDs using Maximum Likelihood Estimators
model.fit(heartDisease, estimator=MaximumLikelihoodEstimator)
print("Nodes of the Model:\n",model.nodes())
print("Edges of the model:\n",model.edges())
print("CPDS for Cholestrol:\n",model.get_cpds('age'))#Cpds for other attributes can be printed in a similar way
#print("Models independencies:\n",model.get_independencies())

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Nodes of the Model:
 ['age', 'trestbps', 'fbs', 'sex', 'exang', 'heartdisease', 'restecg', 'thalach', 'chol']
Edges of the model:
 [('age', 'trestbps'), ('age', 'fbs'), ('trestbps', 'heartdisease'), ('fbs', 'heartdisease'), ('sex', 'trestbps'), ('exang', 'trestbps'), ('heartdisease', 'restecg'), ('heartdisease', 'thalach'), ('heartdisease', 'chol')]
CPDS for Cholestrol:
 +---------+------------+
| age(28) | 0.00340136 |
+---------+------------+
| age(29) | 0.00680272 |
+---------+------------+
| age(30) | 0.00340136 |
+---------+------------+
| age(31) | 0.00680272 |
+---------+------------+
| age(32) | 0.0136054  |
+---------+------------+
| age(33) | 0.00680272 |
+---------+------------+
| age(34) | 0.0136054  |
+---------+------------+
| age(35) | 0.0170068  |
+---------+------------+
| age(36) | 0.0170068  |
+---------+------------+
| age(37) | 0.0272109  |
+---------+------------+
| age(38) | 0.0238095  |
+---------+------------+
| age(39) | 0.037415   |
+---------+------------+
|

In [6]:
#Inferencing with Bayesian Network
# Doing exact inference using Variable Elimination
from pgmpy.inference import VariableElimination
HeartDisease_infer = VariableElimination(model)

# Computing the probability of bronc given smoke.
q = HeartDisease_infer.query(variables=['heartdisease'], evidence={'age': 29}, joint=False) # idakke illi nodu "joint=False" https://github.com/pgmpy/pgmpy/issues/1111
print("\nProbability of gettting heart disease with the evidence age 28:\n ")
print(q['heartdisease'])

q = HeartDisease_infer.query(variables=['heartdisease'], evidence={'chol': 100}, joint=False)
print("Probability of gettting heart disease with the evidence Cholestrol 100:\n ")
print(q['heartdisease'])

Eliminating: fbs: 100%|██████████| 7/7 [00:00<00:00, 393.20it/s]
/usr/local/lib/python3.6/dist-packages/pgmpy/factors/discrete/DiscreteFactor.py:519: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  "Found unknown state name. Trying to switch to using all state names as state numbers"
Eliminating: fbs: 100%|██████████| 7/7 [00:00<00:00, 126.70it/s]


Probability of gettting heart disease with the evidence age 28:
 
+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.6383 |
+-----------------+---------------------+
| heartdisease(1) |              0.3617 |
+-----------------+---------------------+
Probability of gettting heart disease with the evidence Cholestrol 100:
 
+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              1.0000 |
+-----------------+---------------------+
| heartdisease(1) |              0.0000 |
+-----------------+---------------------+
